## From openrefine to cartography

In [ ]:
import pydeck as pdk
import pandas as pd
from shapely.wkt import loads
import numpy as np

### Load the CSV from OpenRefine

In [ ]:
df = pd.read_csv('/Users/carboni/Documents/UNIGE/pynotebook/personal/GLAM/mah_artist_glam.csv')

In [ ]:
df.head()

### Drop the empty value from the column city 

In [ ]:
df = df.dropna(subset=['city'])

In [ ]:
#df = df[~df['city'].str.contains('nan')]

### Count the number of cities

In [ ]:
df['city_number'] = df.groupby(['city'])['city'].transform('count')

### Extract latitude and longitude 
this step is necessary only if you have a unique 'coordinate' column

In [ ]:
df[['latitude', 'longitude']] = df['coordinate location'].str.split(',', expand=True).astype(float)

In [ ]:
df.head()

### Normalize the city count 

In [ ]:
# df['normalized_counts'] = np.log(df['city_number'])

### Create and save the map

In [ ]:
scatter_layer = pdk.Layer(
    'ScatterplotLayer',
    df,
    opacity=0.6,
    get_position='[longitude, latitude]',
#    get_radius='normalized_counts * 5000',
    get_radius='city_number * 1500',
    get_fill_color=[255, 0, 0],  # Red 
    pickable=True,
    stroked=True,
    get_line_color=[255,255,255]
)

In [ ]:
view_state = pdk.ViewState(
    latitude=df['latitude'].mean(),
    longitude=df['longitude'].mean(),
    zoom=3,
)

#### here you can modify the content of the the tooltip for the html map

In [ ]:
tooltip = {
    "html": "<b>{city_number}</b> of artist born in <b>{city}</b>",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

In [ ]:
deck = pdk.Deck(
    layers=[scatter_layer],
    initial_view_state=view_state,
    tooltip=tooltip,
    map_provider="carto",
    map_style="light" #possible here to go for light’, ‘dark’, ‘road’, ‘satellite’, 
    #‘dark_no_labels’, and ‘light_no_labels’. Also possible to use mapbox. To change together with the 
    #parameters on scatter_layer (e.g. opacity!)
)

In [ ]:
deck.to_html(filename='map.html', offline=True, open_browser=False, notebook_display=False)

### Save the dataframe used for the map

In [ ]:
df.to_csv('df.csv', index=False)